In [3]:
from functools import partial
import multiprocessing as mp
import time 
import numpy as np
import os
import multi_gpu as mg

%load_ext autoreload
%autoreload 2

In [4]:
shape = (10000, 500)
steps = 10
NUM_PROC = mp.cpu_count()
NUM_GPUS = 2
manager = mp.Manager()
gpu_queue = manager.Queue(maxsize=NUM_GPUS)
for i in range(NUM_GPUS):
    gpu_queue.put(i)

default_kwargs = dict(func=mg.train, gpu_queue=gpu_queue)
kwargs = [
    dict(**default_kwargs, x=shape, steps=10),
    dict(**default_kwargs, x=shape, steps=30),
    dict(**default_kwargs, x=shape, steps=60),
    dict(**default_kwargs, x=shape, steps=80)
]
print(NUM_GPUS)
results = []
log_results = lambda result: results.append(result)
    
pool = mp.Pool(NUM_GPUS)
jobs = []
for kws in kwargs:
    jobs.append(pool.apply_async(mg.multi_gpu, kwds=kws, callback=log_results))
pool.close()

start = time.time()
while True:
    check_jobs = [j.ready() for j in jobs]
    if not all(check_jobs):
        jobs_left = len(jobs) - sum(check_jobs)
        print('Time: {:4f} Tasks Left: {}'.format(time.time() - start, jobs_left))
        time.sleep(1)
    else:
        break

pool.join()
print([j.successful() for j in jobs])
print("Results: {}".format(results))

2
Time: 0.000184 Tasks Left: 4
Time: 1.001857 Tasks Left: 4
Time: 2.012834 Tasks Left: 4
LogicalDevice(name='/device:GPU:0', device_type='GPU')
Time: 3.013831 Tasks Left: 4
LogicalDevice(name='/device:GPU:0', device_type='GPU')
Time: 4.019897 Tasks Left: 4
Time: 5.023826 Tasks Left: 4
130411  returning:  2.2138309478759766
LogicalDevice(name='/device:GPU:0', device_type='GPU')
Time: 6.028813 Tasks Left: 3
Time: 7.036811 Tasks Left: 3
Time: 8.045818 Tasks Left: 3
Time: 9.046906 Tasks Left: 3
130412  returning:  5.2393951416015625
LogicalDevice(name='/device:GPU:0', device_type='GPU')
Time: 10.055813 Tasks Left: 2
Time: 11.056816 Tasks Left: 2
Time: 12.057810 Tasks Left: 2
Time: 13.058811 Tasks Left: 2
Time: 14.059810 Tasks Left: 2
Time: 15.068835 Tasks Left: 2
130411  returning:  9.803203821182251
Time: 16.070813 Tasks Left: 1
Time: 17.072811 Tasks Left: 1
Time: 18.074809 Tasks Left: 1
Time: 19.076808 Tasks Left: 1
Time: 20.078808 Tasks Left: 1
Time: 21.088808 Tasks Left: 1
130412  retu

In [57]:
shape = (10000, 500)
steps = 10
NUM_PROC = mp.cpu_count()
NUM_GPUS = 2
manager = mp.Manager()
gpu_queue = manager.Queue(maxsize=NUM_GPUS)
for i in range(NUM_GPUS):
    gpu_queue.put(i)

default_kwargs = dict(func=mg.train)
kwargs = [
    dict(**default_kwargs, gpu=0, x=shape, steps=10),
    dict(**default_kwargs, gpu=1, x=shape, steps=30),
    dict(**default_kwargs, gpu=0, x=shape, steps=60),
    dict(**default_kwargs, gpu=1, x=shape, steps=80)
]

results = []
def log_results(result):
    results.append(result)
    
pool = mp.Pool()
jobs = []
for kws in kwargs:
    jobs.append(pool.apply_async(mg.multi_gpu2, kwds=kws, callback=log_results))
pool.close()

start = time.time()
while True:
    check_jobs = [j.ready() for j in jobs]
    if not all(check_jobs):
        jobs_left = len(jobs) - sum(check_jobs)
        print('Time: {:4f} Tasks Left: {}'.format(time.time() - start, jobs_left))
        time.sleep(1)
    else:
        break

pool.join()
print([j.successful() for j in jobs])
print("Results: {}".format(results))

Time: 0.000176 Tasks Left: 4
Time: 1.008430 Tasks Left: 4
Time: 2.019429 Tasks Left: 4
LogicalDevice(name='/device:GPU:0', device_type='GPU')LogicalDevice(name='/device:GPU:0', device_type='GPU')
LogicalDevice(name='/device:GPU:0', device_type='GPU')

LogicalDevice(name='/device:GPU:0', device_type='GPU')
Time: 3.021102 Tasks Left: 4
Time: 4.022437 Tasks Left: 4
Time: 5.023413 Tasks Left: 4
Time: 6.030511 Tasks Left: 4
Time: 7.031408 Tasks Left: 4
Time: 8.041418 Tasks Left: 4
124833  returning:  4.88648533821106
Time: 9.044412 Tasks Left: 4
Time: 10.055404 Tasks Left: 3
Time: 11.058622 Tasks Left: 3
Time: 12.059404 Tasks Left: 3
Time: 13.060412 Tasks Left: 3
124834  returning:  9.964785099029541
Time: 14.061413 Tasks Left: 3
Time: 15.062406 Tasks Left: 2
124835  returning:  11.879923343658447
Time: 16.066963 Tasks Left: 1
Time: 17.076611 Tasks Left: 1
Time: 18.086607 Tasks Left: 1
Time: 19.096605 Tasks Left: 1
Time: 20.105412 Tasks Left: 1
Time: 21.115410 Tasks Left: 1
Time: 22.117419 

In [40]:
# def measure(x, steps, gpu):
#     import tensorflow as tf
#     physical_gpus = tf.config.list_physical_devices('GPU')
#     tf.config.experimental.set_visible_devices(physical_gpus[gpu], 'GPU')
#     tf.config.experimental.set_memory_growth(physical_gpus[gpu], True)

#     x = tf.random.normal(x)
#     x = x @ tf.transpose(x)
#     start = time.time()
#     for i in range(steps):
#         x = x @ tf.transpose(x)
#     _ = x.numpy()
#     end = time.time()
#     print(os.getpid(), ' returning: ', end - start)
#     return end - start

In [25]:
from multiprocessing import Pool, current_process, Queue

NUM_GPUS = 4
PROC_PER_GPU = 2    

queue = Queue()

def foo(filename):
    gpu_id = queue.get()
    try:
        # run processing on GPU <gpu_id>
        ident = current_process().ident
        print('{}: starting process on GPU {}'.format(ident, gpu_id))
        # ... process filename
        print('{}: finished'.format(ident))
    finally:
        queue.put(gpu_id)

# initialize the queue with the GPU ids
for gpu_ids in range(NUM_GPUS):
    for _ in range(PROC_PER_GPU):
        queue.put(gpu_ids)

pool = Pool(processes=PROC_PER_GPU * NUM_GPUS)
files = ['file{}.xyz'.format(x) for x in range(1000)]
with mp.Pool(2) as pool:
    results = pool.starmap_async(mg.measure, args)
    print(results.get())
pool.close()
pool.join()

AttributeError: module 'multi_gpu' has no attribute 'measure'

In [4]:
import tensorflow as tf
from tensorflow.keras import layers

In [5]:
mnist = tf.keras.datasets.mnist
(X_trn, y_trn), (X_tst, y_tst) = mnist.load_data()
X_trn, X_tst = X_trn / 255.0, X_tst / 255.0

In [6]:
class Trainer():
    def __init__(self, X_trn, y_trn, X_tst, y_tst, model, optim, loss, metrics=['accuracy']):
        self.trn_ds = tf.data.Dataset.from_tensor_slices((X_trn.astype(np.float32), 
                                                           y_trn.astype(np.float32))).batch(128)
        
        self.tst_ds = tf.data.Dataset.from_tensor_slices((X_tst.astype(np.float32), 
                                                            y_tst.astype(np.float32))).batch(128)
        self.model = model.compile(optimizer=optim, loss=loss, metrics=metrics)      


    def fiteval(self, epochs=1):
        hist = self.model.fit(self.trn_ds, epochs)

        # Clear tf/keras graph state
        tf.keras.backend.clear_session()
        
        tst_loss, tst_acc = model.evaluate(self.tst_ds, verbose=2)

        # Clear tf/keras graph state
        tf.keras.backend.clear_session()
        
        return tst_loss, tst_acc

In [7]:
from pdb import set_trace

In [8]:
def run_fiteval_multi_gpu(cfgs, n_gpus):
    """ Runs multiple basic training and then testing experiments """
    jobs = []
    results = []
    log_results = lambda result: results.append(result)

    # Build available queue which contains GPUs which are aviable for use
    manager = mp.Manager()
    avail_gpus = manager.Queue(maxsize=n_gpus)
    for i in range(n_gpus):
        avail_gpus.put(i)

    pool = mp.Pool(n_gpus)

    for cfg in cfgs:
        (cfg_name, kwargs), = cfg.items()

        # Build trainer object
        trainer_kws = kwargs.pop('trainer')
        trainer = Trainer(**trainer_kws)

        # Add additional kwargs
        kwargs['gpu_avail'] = avail_gpus
        kwargs['func'] = trainer.fiteval
        set_trace()
        # Initialize processes
        jobs.append(pool.apply_async(tf_multi_gpu, 
                                     kwds=kwargs, 
                                     callback=log_results))

    job_status(jobs)
    
    pool.close()
    pool.join()
    print("Successful Jobs: {} ".format([j.successful() for j in jobs]))
   
    
def tf_multi_gpu(func, avail_gpus, *args, **kwargs):

    # Wrap in try-catch so errors will actually get printed
    try:
        # Get gpu for this process
        gpu = avail_gpus.get()
        
        # Set GPU visible device and set memory growth
        physical_gpus = tf.config.list_physical_devices('GPU')
        tf.config.experimental.set_visible_devices(physical_gpus[gpu], 'GPU')
        tf.config.experimental.set_memory_growth(physical_gpus[gpu], True)
    
        # Run function given selected gpu
        logical_gpus = tf.config.list_logical_devices('GPU')
        with tf.device(logical_gpus[0]):
            results = func(*args, **kwargs)
            
        # Add gpu back to being available 
        avail_gpus.put(gpu)
        
    except Exception as e:
        raise Exception(e)
    
    return results

def job_status(jobs):
    import psutil
    import time
    start = time.time()
    while True:
        check_jobs = [j.ready() for j in jobs]
        if not all(check_jobs):
            jobs_left = len(jobs) - sum(check_jobs)
            print('Time: {:4f} Tasks Left: {} Memory: {}'.format(time.time() - start, 
                                                                jobs_left,
                                                                psutil.virtual_memory()[2]))
            time.sleep(3)
        else:
            break

In [9]:
fc = tf.keras.Sequential(
    [
        layers.Dense(64, activation="relu", input_shape=(28, 28, 1)),
        layers.Dense(64, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ]
)

cnn = tf.keras.Sequential(
    [
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(), 
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation="softmax")
    ]
)

cfgs=[
    {'exp1': {
        'trainer':{
            'X_trn': X_trn,
            'y_trn': y_trn,
            'X_tst': X_tst,
            'y_tst': y_tst,
            'optim': getattr(tf.keras.optimizers, 'Adam')(.01),
            'loss': getattr(tf.keras.losses, 'sparse_categorical_crossentropy'),
            'model': fc
        },
        'epochs': 3
     
    }},
    {'exp2': {
        'trainer':{
            'X_trn': X_trn,
            'y_trn': y_trn,
            'X_tst': X_tst,
            'y_tst': y_tst,
            'optim': tf.keras.optimizers.Adam(.01),
            'loss': tf.keras.losses.sparse_categorical_crossentropy,
            'model': cnn
        },
        'epochs': 3
    }}
]


run_fiteval_multi_gpu(cfgs, n_gpus=1)

> <ipython-input-8-1d0289f30214>(27)run_fiteval_multi_gpu()
     25         set_trace()
     26         # Initialize processes
---> 27         jobs.append(pool.apply_async(tf_multi_gpu, 
     28                                      kwds=kwargs,
     29                                      callback=log_results))

ipdb> c
> <ipython-input-8-1d0289f30214>(25)run_fiteval_multi_gpu()
     23         kwargs['gpu_avail'] = avail_gpus
     24         kwargs['func'] = trainer.fiteval
---> 25         set_trace()
     26         # Initialize processes
     27         jobs.append(pool.apply_async(tf_multi_gpu, 

ipdb> c
Successful Jobs: [False, False] 
